In [1]:
import pandas as pd

### 2019년 9월~11월 자료 불러오기

In [2]:
data1 = pd.read_csv('1.地震リスト20191119부터_20190925까지.csv')
data1

,地震の発生日,地震の発生時刻,震央地名,緯度,経度,深さ,Ｍ,最大震度
0,2019/11/19,18:39:00.6,青森県東方沖,40°41.2′N,142°14.9′E,45 km,3.6,震度２
1,2019/11/18,18:34:37.7,茨城県南部,36°07.2′N,139°51.0′E,48 km,3.5,震度２
2,2019/11/18,08:00:00.7,茨城県北部,36°27.3′N,140°36.0′E,56 km,3.7,震度２
3,2019/11/17,20:05:33.1,伊豆大島近海,34°38.3′N,139°03.1′E,13 km,4.7,震度４
4,2019/11/17,19:20:09.4,茨城県北部,36°28.2′N,140°36.9′E,59 km,3.5,震度２
...,...,...,...,...,...,...,...,...
80,2019/09/28,08:47:10.0,熊本県天草・芦北地方,32°26.5′N,130°28.1′E,13 km,3.5,震度２
81,2019/09/28,02:33:12.7,天草灘,32°21.8′N,129°49.4′E,14 km,4.0,震度３
82,2019/09/27,22:25:11.3,千葉県北東部,35°28.7′N,140°25.8′E,31 km,3.3,震度２
83,2019/09/27,20:11:02.3,宮城県沖,38°55.0′N,141°55.9′E,51 km,3.8,震度２


### NULL 처리

In [ ]:
data1.isnull().sum()

### 데이터 피처링

In [3]:
data1.columns = ['발생일', '발생시각', '위치', '위도', '경도', '깊이(km)', '규모', '진도']

In [4]:
data1['발생시각'] = data1['발생일'] + data1['발생시각']

In [5]:
data1 = data1.drop('발생일', axis=1)

In [6]:
def clean_location(data3):
    # "°"와 "N"을 빈 문자열로 대체하여 제거
    cleaned_location = data3.replace("°", ".").replace("′N", "")
    return cleaned_location

# 'Location' 컬럼에 apply() 함수 적용
data1['위도'] = data1['위도'].apply(clean_location)

In [7]:
def clean_location(data):
    # "°"와 "N"을 빈 문자열로 대체하여 제거
    cleaned_location = data.replace("°", ".").replace("′E", "")
    return cleaned_location

# 'Location' 컬럼에 apply() 함수 적용
data1['경도'] = data1['경도'].apply(clean_location)

In [8]:
def clean_location(data):
    # "°"와 "N"을 빈 문자열로 대체하여 제거
    cleaned_location = data.replace("震度", "")
    return cleaned_location

# 'Location' 컬럼에 apply() 함수 적용
data1['진도'] = data1['진도'].apply(clean_location)

In [9]:
def clean_location(data):
    # "°"와 "N"을 빈 문자열로 대체하여 제거
    parts = data.split('.')
    cleaned_data = parts[0] + '.' + parts[1]
    return cleaned_data

# 'Location' 컬럼에 apply() 함수 적용
data1['위도'] = data1['위도'].apply(clean_location)
data1['경도'] = data1['경도'].apply(clean_location)

In [10]:
def clean_location(data):
    # "°"와 "N"을 빈 문자열로 대체하여 제거
    cleaned_location = data.replace("弱", "").replace("強", "")
    return cleaned_location

# 'Location' 컬럼에 apply() 함수 적용
data1['진도'] = data1['진도'].apply(clean_location)

In [11]:
data1 = data1.drop(data1[data1['규모'] == '不明'].index)

In [12]:
def clean_location(data):
    # "°"와 "N"을 빈 문자열로 대체하여 제거
    cleaned_location = data.replace("km", "")
    return cleaned_location

# 'Location' 컬럼에 apply() 함수 적용
data1['깊이(km)'] = data1['깊이(km)'].apply(clean_location)

In [13]:
data1['발생시각'] = data1['발생시각'].str[:10]

In [14]:
# 컬럼 이름을 변경할 새로운 이름 지정
new_column_names = {'발생시각': 'OT',
                    '위치': 'SPOT',
                   '위도':'LAT',
                   '경도':'LON',
                   '깊이(km)':'DEP',
                   '규모':'MT',
                   '진도':'SC'}

# 컬럼 이름 변경
data1.rename(columns=new_column_names, inplace=True)

In [15]:
data1['OT'] = pd.to_datetime(data1['OT'])
data1 = data1.astype({'LAT':'float'})
data1 = data1.astype({'LON':'float'})
data1 = data1.astype({'MT':'float'})
data1 = data1.astype({'SC':'float'})
data1 = data1.astype({'DEP':'float'})

In [16]:
min_value = 121
max_value = 141

# 조건을 만족하는 행을 선택하고 삭제
data1 = data1[(data1['LON'] >= min_value) & (data1['LON'] <= max_value)]

In [17]:
new_column_order = ['OT', 'MT', 'DEP', 'SC', 'LAT', 'LON', 'SPOT']
data1 = data1[new_column_order]
data1

,OT,MT,DEP,SC,LAT,LON,SPOT
1,2019-11-18,3.5,48.0,2.0,36.07,139.51,茨城県南部
2,2019-11-18,3.7,56.0,2.0,36.27,140.36,茨城県北部
3,2019-11-17,4.7,13.0,4.0,34.38,139.03,伊豆大島近海
4,2019-11-17,3.5,59.0,2.0,36.28,140.36,茨城県北部
5,2019-11-17,4.6,28.0,2.0,28.48,129.54,奄美大島近海
7,2019-11-15,2.5,10.0,2.0,37.21,138.48,新潟県中越地方
8,2019-11-15,3.7,62.0,2.0,36.01,140.04,茨城県南部
10,2019-11-13,2.7,0.0,2.0,29.37,129.44,トカラ列島近海
11,2019-11-13,3.2,8.0,2.0,34.51,135.37,大阪府北部
13,2019-11-12,3.5,7.0,3.0,36.03,137.34,岐阜県飛騨地方


### 19년 11월~ 22년 9월까지 자료 불러오기

In [21]:
data2 = pd.read_csv('Japan_2019_2022.csv', encoding='cp949')

In [22]:
data2

,OT,MT,DEP,SC,LAT,LON,SPOT
0,2022-09-20 00:00:00,3.1,12,2,37.31,137.17,石川?能登地方
1,2022-09-20 00:00:00,4.4,70,3,36.20,139.49,?木?南部
2,2022-09-18 00:00:00,5.7,24,3,26.20,125.57,沖?本島北西沖
3,2022-09-18 00:00:00,6.0,8,2,26.20,125.55,沖?本島北西沖
4,2022-09-18 00:00:00,4.7,51,3,35.51,140.38,千葉?北東部
...,...,...,...,...,...,...,...
1367,2019-11-23 00:00:00,4.4,27,2,31.56,131.52,日向灘
1368,2019-11-22 00:00:00,5.2,24,3,31.56,131.54,日向灘
1369,2019-11-22 00:00:00,4.5,45,3,36.04,139.53,茨城?南部
1370,2019-11-20 00:00:00,2.9,10,2,37.03,138.02,新潟?上越地方


### 데이터 병합

In [23]:
data3 = pd.concat([data1, data2], axis=0)
data3

,OT,MT,DEP,SC,LAT,LON,SPOT
1,2019-11-18 00:00:00,3.5,48.0,2.0,36.07,139.51,茨城県南部
2,2019-11-18 00:00:00,3.7,56.0,2.0,36.27,140.36,茨城県北部
3,2019-11-17 00:00:00,4.7,13.0,4.0,34.38,139.03,伊豆大島近海
4,2019-11-17 00:00:00,3.5,59.0,2.0,36.28,140.36,茨城県北部
5,2019-11-17 00:00:00,4.6,28.0,2.0,28.48,129.54,奄美大島近海
...,...,...,...,...,...,...,...
1367,2019-11-23 00:00:00,4.4,27.0,2.0,31.56,131.52,日向灘
1368,2019-11-22 00:00:00,5.2,24.0,3.0,31.56,131.54,日向灘
1369,2019-11-22 00:00:00,4.5,45.0,3.0,36.04,139.53,茨城?南部
1370,2019-11-20 00:00:00,2.9,10.0,2.0,37.03,138.02,新潟?上越地方


In [25]:
data3.to_csv('Japan_2019_2022(!).csv', index=False)

In [26]:
data3['OT'] = pd.to_datetime(data3['OT']).dt.strftime('%Y-%m-%d')
data3

,OT,MT,DEP,SC,LAT,LON,SPOT
1,2019-11-18,3.5,48.0,2.0,36.07,139.51,茨城県南部
2,2019-11-18,3.7,56.0,2.0,36.27,140.36,茨城県北部
3,2019-11-17,4.7,13.0,4.0,34.38,139.03,伊豆大島近海
4,2019-11-17,3.5,59.0,2.0,36.28,140.36,茨城県北部
5,2019-11-17,4.6,28.0,2.0,28.48,129.54,奄美大島近海
...,...,...,...,...,...,...,...
1367,2019-11-23,4.4,27.0,2.0,31.56,131.52,日向灘
1368,2019-11-22,5.2,24.0,3.0,31.56,131.54,日向灘
1369,2019-11-22,4.5,45.0,3.0,36.04,139.53,茨城?南部
1370,2019-11-20,2.9,10.0,2.0,37.03,138.02,新潟?上越地方


In [27]:
data3.to_csv('Japan_2019_2022(!).csv', index=False)